# Pretrained Model, regression (target MEM-Score), Elastic Net


In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'conn' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = 'conn')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [5]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,1_15,1_16,1_17,1_18,1_19,1_20,1_21,1_22,1_23,1_24,1_25,1_26,1_27,1_28,1_29,1_30,...,237_243,237_244,237_245,237_246,238_239,238_240,238_241,238_242,238_243,238_244,238_245,238_246,239_240,239_241,239_242,239_243,239_244,239_245,239_246,240_241,240_242,240_243,240_244,240_245,240_246,241_242,241_243,241_244,241_245,241_246,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,1.430655,0.958171,0.51861,0.591034,0.227465,0.787522,0.776009,0.798374,0.999554,0.954094,0.837725,0.197717,0.270467,0.48662,0.380078,0.606075,0.605735,0.444788,0.29387,0.379651,0.481484,0.318099,0.13934,0.727726,0.699307,-0.207808,-0.32161,0.592996,0.621801,...,0.35924,0.56932,0.145873,0.152402,-0.075421,0.039472,-0.0109,0.147892,0.19681,0.327893,-0.064144,0.058777,0.327169,0.295942,0.325517,0.226479,-0.045583,0.194034,0.246963,0.798343,0.968418,0.716178,0.343208,-0.034113,0.125901,0.587815,0.670493,0.249516,-0.032901,0.216782,0.655094,0.547549,-0.256346,0.079905,0.504077,0.031435,0.180466,0.095308,-0.062235,0.210861
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.700115,0.815505,0.480916,0.274557,0.048538,0.719735,0.46748,0.521357,0.449369,0.952166,0.511867,0.49259,-0.023851,0.647918,0.305384,0.749082,0.254135,0.586528,0.076689,0.323022,0.080136,0.8353,0.366546,0.774213,0.567104,0.323971,0.095188,0.100919,0.375862,...,0.57138,0.541166,-0.276125,-0.038445,-0.105363,0.090075,-0.160635,0.673594,0.663931,0.624048,-0.128801,-0.008332,0.532339,0.520765,-0.029643,0.181075,-0.005707,0.577066,0.498839,0.251391,0.396573,0.401372,0.27947,0.280546,0.154698,0.095783,0.095389,-0.211978,0.388244,0.257077,0.795011,0.624784,-0.115689,0.055597,0.684795,0.212303,0.193802,-0.117644,-0.092837,0.621967
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.75266,0.231123,0.149763,0.450447,0.185762,0.599392,0.156499,0.782779,0.427059,0.327918,0.181461,0.017913,-0.112979,0.183447,0.120085,0.279274,0.116219,0.322515,0.155501,-0.090248,-0.125024,0.1874,-0.05036,0.586258,0.491451,-0.030206,-0.222713,0.031067,-0.160859,...,0.849411,0.71605,-0.025114,0.291594,-0.051196,0.082976,-0.24298,0.118436,0.250335,0.286292,-0.198465,0.158462,0.279858,0.002769,0.119833,0.023414,0.007263,0.150606,0.057617,-0.023502,0.552398,-0.001833,0.141841,0.069829,0.435568,0.243923,0.134239,0.033118,0.132755,-0.016676,0.209853,0.166607,0.056147,0.159351,0.721864,0.107519,0.329301,0.055294,0.39387,0.269579
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,1.290996,0.583473,0.46487,0.323327,0.529798,0.795451,0.795769,0.789307,1.014339,0.621851,0.585669,0.10045,0.008213,0.794467,0.675788,0.467286,0.203388,0.035303,0.006168,0.150005,-0.085235,0.458474,0.159425,0.474333,0.528865,-0.042296,-0.063538,0.173261,0.101468,...,0.641677,0.646977,0.232723,0.434119,0.337657,0.255091,0.217298,0.342232,0.20307,0.175855,0.38717,0.368688,0.48521,0.451154,0.244162,0.324582,0.086557,0.302443,0.498218,1.044369,0.963662,0.426679,0.383891,0.14173,0.336913,0.872448,0.527014,0.422302,-0.065849,0.158495,0.547318,0.724798,0.205874,0.193587,0.855264,0.085781,0.333715,0.231567,0.28501,0.780887
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.836155,0.605629,0.507591,0.469744,0.358818,0.451134,0.464819,0.433019,0.389306,0.889666,0.421906,0.109799,0.293217,0.383408,0.389929,0.356466,0.308653,0.337839,0.192534,0.155588,0.151168,0.499594,0.34374,0.798282,0.618485,-0.099827,0.079852,0.428427,0.161856,...,0.902319,0.459896,-0.102687,0.205158,0.415207,0.310483,0.426834,0.348153,0.548878,0.175564,0.095634,0.326211,0.085041,0.339179,0.337291,0.768877,0.343013,0.575715,0.425439,0.509784,0.772538,0.37473,0.485332,-0.11383,0.2927,0.523292,0.551723,0.250376,0.11494,0.021934,0.901057,0.547402,-0.037584,0.089426,0.509276,0.122451,0.308097,0.330322,0.360479,0.404907
...,...,...,...,...,...,...,...,...,...,...,...,...,

In [6]:
classification = False
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "prmdiag", "Apoe", "IDs"]
target = "MEM_score"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [7]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,1_15,1_16,1_17,1_18,1_19,1_20,1_21,1_22,1_23,1_24,1_25,1_26,1_27,1_28,1_29,1_30,1_31,1_32,1_33,1_34,1_35,1_36,1_37,...,237_243,237_244,237_245,237_246,238_239,238_240,238_241,238_242,238_243,238_244,238_245,238_246,239_240,239_241,239_242,239_243,239_244,239_245,239_246,240_241,240_242,240_243,240_244,240_245,240_246,241_242,241_243,241_244,241_245,241_246,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
721,0.973914,-0.500490,0.690935,-0.851425,-0.122098,-1.531785,0.976066,0.661138,1.269696,-2.614527,-1.694126,-0.670141,-1.772189,-0.110504,0.825215,1.268021,1.786742,-3.724806,-0.680454,0.937891,0.894626,-0.729884,-0.912806,-2.249520,-0.828996,-0.168308,1.612221,-0.120642,1.079719,0.918877,1.604153,-0.049948,0.839497,-0.514593,-0.902969,2.343392,1.481919,1.117674,2.051999,0.803472,...,0.359006,1.014876,0.949901,-0.014662,1.272312,0.317250,-0.249491,0.343299,0.298301,0.105202,1.260926,0.810701,2.333107,1.301927,1.509657,1.964753,0.938668,0.827549,1.806141,1.712692,1.333840,1.678752,0.696907,0.705716,2.214916,1.277822,2.137067,0.764166,1.002103,1.713692,1.002597,0.260196,0.601955,1.779217,0.470700,1.935919,2.236521,0.275453,0.833528,0.009996
335,0.281476,-0.174161,0.690935,0.819703,-0.007078,-0.029512,0.476988,0.818049,0.361422,0.969541,-0.162317,-1.423914,-1.032428,0.071433,-0.497982,-0.249126,-0.117372,1.564791,0.883258,0.711079,0.464384,0.864615,1.542760,1.003325,0.898792,0.320931,0.134794,0.408480,1.205183,1.172799,1.058037,1.929732,0.024746,1.462583,0.281344,0.267339,0.268949,1.443045,0.763198,-0.123955,...,0.883330,0.519481,0.150067,0.360927,1.190270,0.509891,1.524833,1.873902,0.690332,-0.210016,0.272708,0.271031,-0.354722,0.231439,0.277871,0.411655,-0.378475,0.635808,0.342570,0.022373,-0.642483,-1.990294,-0.332900,1.634753,1.132793,0.570959,-0.315151,-0.966179,0.887855,0.592579,-0.430766,-0.026134,0.570157,0.241247,0.027234,-1.164211,-1.394620,-1.252578,-1.010716,2.087324
590,0.19615,-0.337326,-1.041167,0.151252,-0.481527,1.687979,0.565462,0.838781,0.781065,0.545835,-0.403791,-1.075546,-0.957436,1.961058,1.915439,0.998178,1.058422,-0.706027,-0.831403,1.191239,0.407570,0.733261,-0.808931,0.532327,-1.359885,0.471128,0.217607,1.417302,0.922625,-0.758670,-1.549752,0.445365,-0.165683,0.348808,0.167238,0.489681,0.984819,1.105288,0.803533,1.673348,...,1.396491,0.754878,-0.489618,-0.393252,-1.051763,-0.175205,2.209444,0.989863,1.413441,1.368867,-1.112484,-1.021955,-0.717231,-0.810576,-1.423875,-0.317001,0.025458,1.237133,1.970579,0.838948,0.477890,0.789399,0.613653,-0.099594,-1.309574,0.598961,0.888011,1.239900,-0.678776,-0.306281,1.877342,0.268505,-1.647605,-1.449390,0.548717,-0.547300,-0.258585,0.883507,0.703147,1.573589
7,-1.815235,0.641659,0.690935,-0.851425,2.655245,0.882687,2.279814,0.766425,1.038359,-0.670575,-0.392875,1.145849,-0.679502,3.514365,3.065412,0.344934,2.160046,0.608450,1.803486,-0.592012,1.548163,1.406027,0.592338,-1.962294,-0.638372,0.790222,2.087073,0.623757,2.202596,-0.598880,0.505095,-0.556500,1.579389,0.299918,0.650931,-0.157013,0.269060,-3.230056,1.056858,-1.594159,...,-0.137105,-1.688827,3.037123,1.940493,0.165051,-0.552591,-1.059234,-0.598547,-1.135289,-1.744019,3.284359,2.772862,1.962786,6.493295,4.179222,4.563582,-1.283257,1.927481,0.970069,0.558876,3.581822,1.585111,-0.295215,1.832087,0.688819,1.653585,2.470623,-1.549632,2.578974,1.674262,2.125882,-0.155262,2.894775,1.899853,-2.362812,3.250795,2.189296,-1.785543,-1.410756,3.602463
567,0.676812,-0.174161,0.690935,-0.517199,0.819659,0.338740,-0.456225,-1.443777,-0.783160,1.719091,1.183092,-0.757579,0.049301,-1.308864,-0.353527,-0.645237,-0.461896,-0.298013,0.545756,1.432914,2.353812,1.047781,-0.213372,2.726632,0.873804,2.042988,2.240166,-0.131490,1.218354,1.441060,1.059738,0.053382,0.087495,0.630391,0.223934,-0.208591,-0.550508,0.112259,0.065899,0.076348,...,0.204357,-0.217420,-0.340334,1.579144,1.113582,0.113602,1.148969,-0.170976,0.804259,-0.42

# Train Model

In [8]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, classification = False, 
                          n_alphas_linreg = 10, 
                          l1_ratios_linreg= np.linspace(0.01,  1, 11).tolist(),
                          cv_linreg = 5)               

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.085e-01, tolerance: 5.380e-02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.172e-02, tolerance: 5.380e-02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

In [11]:
predictions = model.predict(Xtest)
mean_squared_error(ytest, predictions)

0.5970362458270458

In [12]:
r2_score(ytest, predictions)

0.39021395905623935

In [13]:
model.l1_ratio_

0.109

In [14]:
model.alpha_

0.36219673288437215

# Save Model

In [15]:
import pickle

In [16]:
filename ='pretrained_elnet_regr.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)